In [2]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import operator

In [3]:
#Exploring some basic elements spanning all players
basic = pd.read_csv('Basic_Stats.csv', encoding='utf8')
playerdict = {}
qualities = ('Name', 'Experience', 'Height (inches)', 'Weight (lbs)', 'Position')
for i in range(len(basic['Current Status'])):
    if basic['Current Status'][i] == 'Active':
        tor = {}
        for a in qualities:
            tor[a] = basic[a][i]
        playerdict[basic['Player Id'][i]] = tor

## Exploring passing scores
![passing.png](passing.png)


In [4]:
#Exploring passing
'''
· TD Pass = 4pts
· Every 25 passing yards = 1pts
· 2pt Passing Conversion = 2pts
· Interceptions Thrown = -2pts
'''
passes = pd.read_csv('Career_Stats_Passing.csv', encoding='utf8')
passdict = {}
qualities = ('Games Played', 'Passes Attempted', 'Completion Percentage', 'Pass Attempts Per Game', 'TD Passes', 'Percentage of TDs per Attempts', 'Passes Longer than 20 Yards', 'Ints', 'Int Rate', 'Passing Yards Per Attempt')
for i in range(len(passes['Team'])):
    if passes['Player Id'][i] in playerdict:
        tor = {}
        for a in qualities:
            tor[a] = float(passes[a][i]) if passes[a][i] != '--' else 0
        if passes['Player Id'][i] in passdict:
            s = passdict[passes['Player Id'][i]]
            d = {}
            for key, val in s.items():
                if key != 'times':
                    passdict[passes['Player Id'][i]][key] = ((s[key]*s['times'])+tor[key])/(s['times']+1)
            passdict[passes['Player Id'][i]]['times'] += 1
        else:
            tor['times'] = 1
            passdict[passes['Player Id'][i]] = tor
            passdict[passes['Player Id'][i]]['times'] = 1
pass_scores = {}
for key, val in passdict.items():
    g, att, p_c, _, TD, p_TD, p_20, i, p_i, py = qualities
    p = passdict[key]
    pass_scores[key] = ((p[att]*p[py]*p[p_c])/(p[g]*100)) + (p[TD]*p[p_TD]*4/(p[g]*100)) + (2*p[p_20]*p[p_c]/(p[g]*100)) - (2*p[i]*p[p_i]/(p[g]*100))
top = sorted(pass_scores.items(), key=operator.itemgetter(1))
top.reverse()
positions = {}
for a in top:
    if playerdict[a[0]]['Position'] not in positions:
        positions[playerdict[a[0]]['Position']] = 1
    else:
        positions[playerdict[a[0]]['Position']] += 1
print(positions)

top_by_position = {}

for key in positions.keys():
    top_by_position[key] = []

for a in top:
    pos = playerdict[a[0]]['Position']
    #Limit it to 12 since there's only 12 people in my league
    if len(top_by_position) < 12:
        top_by_position[pos].append(a)

for key, val in top_by_position.items():
    print('Top', key+'s for passing')
    for a in val[:12]:
        print(playerdict[a[0]]['Position'], '-', playerdict[a[0]]['Name'], '\nScore:', a[1])
    print('')

{'QB': 76, 'WR': 35, 'P': 14, 'RB': 11, 'SS': 2, 'NT': 1, 'K': 6, 'CB': 1, 'TE': 1, 'ILB': 1}
Top QBs for passing
QB - Brees, Drew 
Score: 190.9470415973713
QB - Ryan, Matt 
Score: 176.25137245696396
QB - Cousins, Kirk 
Score: 175.09741808695662
QB - Stafford, Matthew 
Score: 168.76040173451838
QB - Roethlisberger, Ben 
Score: 167.9310715496562
QB - Luck, Andrew 
Score: 163.75400502857138
QB - Prescott, Dak 
Score: 159.10449999999997
QB - Brady, Tom 
Score: 157.93311046676294
QB - Rivers, Philip 
Score: 156.51541969428004
QB - Winston, Jameis 
Score: 155.69698125000002
QB - Tannehill, Ryan 
Score: 155.67245200000002
QB - Dalton, Andy 
Score: 155.0574910394265

Top WRs for passing
WR - Pryor, Terrelle 
Score: 9.949116235294117
WR - McEvoy, Tanner 
Score: 3.2142857142857144
WR - Sanu, Mohamed 
Score: 1.1154555555555556
WR - Snead, Willie 
Score: 0.5166666666666667
WR - Kerley, Jeremy 
Score: 0.21975308641975308
WR - Lee, Marqise 
Score: 0.10826210826210825
WR - Meredith, Cameron 
Score: 

## Exploring Rushing
![rushing.png](rushing.png)

In [5]:
#Exploring rushing
'''
· TD Rush = 6pts
· Every 10 rushing yards = 1pt
· 2pt Rushing Conversion = 2pts
'''
rushes = pd.read_csv('Career_Stats_Rushing.csv', encoding='utf8')
rushdict = {}
qualities = ('Games Played', 'Rushing Yards', 'Rushing TDs')
for i in range(len(rushes['Team'])):
    if rushes['Player Id'][i] in playerdict:
        tor = {}
        for a in qualities:
            tor[a] = float(str(rushes[a][i]).replace(',','')) if rushes[a][i] != '--' else 0
        if rushes['Player Id'][i] in rushdict:
            s = rushdict[rushes['Player Id'][i]]
            d = {}
            for key, val in s.items():
                if key != 'times':
                    rushdict[rushes['Player Id'][i]][key] = ((s[key]*s['times'])+tor[key])/(s['times']+1)
            rushdict[rushes['Player Id'][i]]['times'] += 1
        else:
            tor['times'] = 1
            rushdict[rushes['Player Id'][i]] = tor
            rushdict[rushes['Player Id'][i]]['times'] = 1
rush_scores = {}
for key, val in rushdict.items():
    g, y, TD = qualities
    p = rushdict[key]
    rush_scores[key] = (60*p[TD]+p[y])/(10*p[g])
top = sorted(rush_scores.items(), key=operator.itemgetter(1))
top.reverse()
positions = {}
for a in top:
    if playerdict[a[0]]['Position'] not in positions:
        positions[playerdict[a[0]]['Position']] = 1
    else:
        positions[playerdict[a[0]]['Position']] += 1
print(positions)

top_rush_by_position = {}

for key in positions.keys():
    top_rush_by_position[key] = []

for a in top:
    pos = playerdict[a[0]]['Position']
    #Limit it to 12 since there's only 12 people in my league
    top_rush_by_position[pos].append(a)

for key, val in top_rush_by_position.items():
    print('Top', key+'s for rushing')
    for a in val[:12]:
        print(playerdict[a[0]]['Position'], '-', playerdict[a[0]]['Name'], '\nScore:', a[1])
    print('')

{'RB': 126, 'QB': 73, 'WR': 120, 'FB': 15, 'DE': 2, 'NT': 1, 'TE': 26, 'SS': 5, 'P': 24, 'CB': 4, 'K': 2, 'FS': 2, 'DB': 2}
Top RBs for rushing
RB - Elliott, Ezekiel 
Score: 16.873333333333335
RB - Peterson, Adrian 
Score: 14.282113821138212
RB - Bell, Le'Veon 
Score: 11.925531914893616
RB - Howard, Jordan 
Score: 11.153333333333334
RB - Murray, DeMarco 
Score: 10.827380952380953
RB - McCoy, LeSean 
Score: 10.729914529914529
RB - Lynch, Marshawn 
Score: 10.670866141732283
RB - Johnson, David 
Score: 10.1875
RB - Gurley, Todd 
Score: 10.175862068965516
RB - Martin, Doug 
Score: 9.836842105263159
RB - Gore, Frank 
Score: 9.725
RB - Hill, Jeremy 
Score: 9.568085106382979

Top QBs for rushing
QB - Newton, Cam 
Score: 6.931182795698926
QB - Brissett, Jacoby 
Score: 4.766666666666667
QB - Taylor, Tyrod 
Score: 4.520930232558139
QB - Wilson, Russell 
Score: 4.33625
QB - Hogan, Kevin 
Score: 4.125
QB - Prescott, Dak 
Score: 4.0125
QB - Luck, Andrew 
Score: 3.26
QB - Smith, Geno 
Score: 3.23333

## Exploring Reception

This is basically the same as receiving with some nominal changes

In [6]:
#Exploring reception
'''
· TD Reception = 6pts
· Every 10 receiving yards = 1pt
· 2pt Receiving Conversion = 2pts
'''
receptions = pd.read_csv('Career_Stats_Receiving.csv', encoding='utf8')
recdict = {}
qualities = ('Games Played', 'Receiving Yards', 'Receiving TDs')
for i in range(len(receptions['Team'])):
    if receptions['Player Id'][i] in playerdict:
        tor = {}
        for a in qualities:
            tor[a] = float(str(receptions[a][i]).replace(',','')) if receptions[a][i] != '--' else 0
        if receptions['Player Id'][i] in recdict:
            s = recdict[receptions['Player Id'][i]]
            d = {}
            for key, val in s.items():
                if key != 'times':
                    recdict[receptions['Player Id'][i]][key] = ((s[key]*s['times'])+tor[key])/(s['times']+1)
            recdict[receptions['Player Id'][i]]['times'] += 1
        else:
            tor['times'] = 1
            recdict[receptions['Player Id'][i]] = tor
            recdict[receptions['Player Id'][i]]['times'] = 1
rec_scores = {}
for key, val in recdict.items():
    g, y, TD = qualities
    p = recdict[key]
    rec_scores[key] = (60*p[TD]+p[y])/(10*p[g])
top = sorted(rec_scores.items(), key=operator.itemgetter(1))
top.reverse()
positions = {}
for a in top:
    if playerdict[a[0]]['Position'] not in positions:
        positions[playerdict[a[0]]['Position']] = 1
    else:
        positions[playerdict[a[0]]['Position']] += 1
print(positions)

top_rec_by_position = {}

for key in positions.keys():
    top_rec_by_position[key] = []

for a in top:
    pos = playerdict[a[0]]['Position']
    #Limit it to 12 since there's only 12 people in my league
    top_rec_by_position[pos].append(a)

for key, val in top_rec_by_position.items():
    print('Top', key+'s for receiving')
    for a in val[:12]:
        print(playerdict[a[0]]['Position'], '-', playerdict[a[0]]['Name'], '\nScore:', a[1])
    print('')

{'WR': 202, 'TE': 115, 'RB': 125, 'FB': 16, 'QB': 27, 'DE': 4, 'DB': 2, 'CB': 3, 'FS': 3, 'LB': 1}
Top WRs for receiving
WR - Beckham, Odell 
Score: 14.469767441860464
WR - Jones, Julio 
Score: 12.670886075949367
WR - Green, A.J. 
Score: 11.715116279069766
WR - Evans, Mike 
Score: 11.299999999999999
WR - Brown, Antonio 
Score: 11.264356435643567
WR - Thomas, Michael 
Score: 11.18
WR - Bryant, Dez 
Score: 10.970103092783503
WR - Thomas, Demaryius 
Score: 10.71683168316832
WR - Bryant, Martavis 
Score: 10.257142857142858
WR - Fitzgerald, Larry 
Score: 10.212376237623761
WR - Marshall, Brandon 
Score: 10.168263473053893
WR - Robinson, Allen 
Score: 9.883333333333333

Top TEs for receiving
TE - Gronkowski, Rob 
Score: 11.5625
TE - Graham, Jimmy 
Score: 9.352380952380953
TE - Gates, Antonio 
Score: 8.750980392156862
TE - Reed, Jordan 
Score: 8.265217391304347
TE - Kelce, Travis 
Score: 7.555102040816327
TE - Eifert, Tyler 
Score: 7.272972972972973
TE - Witten, Jason 
Score: 7.02600896860986